# Práctica de Neuroevolución

* Pablo Alonso López.
* Marcos Cedenilla.


Resolución de los juegos Lunar-Lander en su versión discreta y Bipedal Walker en su versión normal, redes entrenadas con el software SALGA-3.8.6

Antes de pasar a la resolución hemos de adjuntar el código de nuestra implementación de redes neuronales, modificada levemente desde la práctica anterior.

In [2]:
import numpy as np

# Funciones de activación usadas para la red neuronal, con sus respectivas derivadas

class ReLu():
    
    def function(x):
        return np.where(x>0, x, np.zeros(x.shape))
    
class Sigmoid():
    
    def function(x):
        return 1 / (1 + np.exp(-x))
    
class Tanh():
    
    def function(x):
        return np.tanh(x)
    
    def derivative(x):
        return 1 - np.square(Tanh.tanh(x))

In [3]:
# Implementación de una capa, con funciones para hallar el output de la capa y la suma ponderada (z) de los pesos con la entrada

class Layer():
    
    def __init__(self, n_neurons, input_shape, activation = "sigmoid"):
        self.input_shape = input_shape
        self.n_neurons = n_neurons
        self.nw = input_shape*n_neurons
        self.nb = n_neurons 
        
        if activation.lower() == "relu":
            self.activation = ReLu    
        elif activation.lower() == "tanh":
            self.activation = Tanh      
        else:
            self.activation = Sigmoid       
            
    def output(self, x):
        return self.activation.function(x@self.weights + self.bias)
    
    def z(self, x):
        return x@self.weights + self.bias

In [4]:
# Implementación de la red neuronal ya implementada, esta clase se crea debido a la llamda de MLP, definida posteriormente

class MLP_compiled():
    def __init__(self, topology, layers, activation, f_input_shape):
        self.__layers = layers
        self.__activation = activation
        self.__topology = topology
        self.__f_input_shape = f_input_shape
        
    def layers(self):
        return self.__layers
    
    # Método para asignar los pesos en base a un cromosoma, el cual tendrá forma de vector 
    
    def from_chromosome(self, chromosome):
        chromosome = np.array(chromosome)
        acum = 0
        # Se recorre el cromosoma por saltos, para ir guardando los pesos y bias en cada capa de la red
        for layer in self.layers():
            
            layer.weights = chromosome[acum:acum+ layer.nw].reshape(layer.input_shape, layer.n_neurons)
            acum += layer.nw
            layer.bias = chromosome[acum: acum+ layer.nb].reshape(1, layer.n_neurons)
            acum += layer.nb
            
    # Método para obtener el tamaño de cromosoma
        
    def chromsize(self):
        cs = 0
        
        for layer in self.layers():
            cs+= layer.nw
            cs+= layer.nb
            
        return cs
    
    def forward(self, x): # Función para hallar la salida de la red para una muestra, 
                          # orientada a entrenar la red, se almacenan por ello las sumas ponderadas y salidas de cada capa
        self.__outputs = []
        self.__zs = []
        x = x.reshape(1,x.size)
        for i in self.__layers:
            self.__outputs.append(x)
            self.__zs.append(i.z(x))
            x = i.output(x)
        return x.reshape(x.size,-1)

In [5]:
# Implementación de la clase perceptrón multicapa, para crear la topologia de la red

class MLP():
    
    def __init__(self, f_input_shape):
        self.__topology = dict()
        self.__activation = list()
        self.__layers = list()
        self.__f_input_shape = f_input_shape
        
    def add_layer(self, n_neurons, activation = "sigmoid"): # Función para añadir capas, no se necesita en ningún caso añadir el input shape
                                                            # La forma de la primera entrada, se le indicará en el constructor de la red, el resto se calcularán en función
                                                            # de la forma de la salida anterior
        
        if len(self.__layers) == 0:
            self.__layers.append(Layer(n_neurons, self.__f_input_shape, activation))
            self.__topology["Input Layer"] = n_neurons
            self.__activation.append(str(self.__layers[-1].activation()))
        else:
            self.__layers.append(Layer(n_neurons, self.__layers[-1].n_neurons, activation))
            self.__topology["Layer " + str(len(self.__layers)-1)] = n_neurons
            self.__activation.append(str(self.__layers[-1].activation()))
            
    def compile(self):
        return MLP_compiled(self.__topology, self.__layers, self.__activation, self.__f_input_shape)

## Lunar Lander

En la siguiente celda se adjuntará el código que se uso para entrenar la red con salga.

In [6]:
import numpy as np
import gymnasium as gym

model = MLP(8)

# Conseguimos superar Lunar-Lander con un perceptrón simple, una sola capa de 4 salidas

model.add_layer(4,'sigmoid')

# Las salidas de una sigmoid, van entre 0 y 1, se cogerá la acción de las 4 que más valor tenga

model = model.compile()

n_sims = 4 # número de simulaciones por red

env = gym.make("LunarLander-v2", render_mode=None)

def run(model):

    observation, info = env.reset()
    racum = 0
    while True:
        # Seleccionamos la acción
        action = np.argmax(model.forward(observation))
        observation, reward, terminated, truncated, info = env.step(action)
        
        racum += reward

        if terminated or truncated:
            # Normalización de la recompensa
            r = (racum+200) / 500
            return r
        
        
def fitness(chromosome):
    
    model.from_chromosome(chromosome)
    
    scores = []
    
    # Obtenemos como fitness el score medio obtenido en n_sims partidas
    for i in range(n_sims):
        scores.append(run(model))
        
    fit = np.mean(scores)
    
    return fit


parameters = { 'alphabet':[-3, 3], 'target': 1, 'type':'floating', 'elitism':True, 'norm':True, 'chromsize':model.chromsize(), 'pmut':2/model.chromsize() }

Del entrenamiento anterior con SALGA obtenemos los siguientes pesos, se demostrará su validez con un ejemplo renderizado

In [7]:
params = [1.5002627772298862, 0.993280412677656, 0.7445702653578861, -0.9118845760852753, -0.1085073530174191, -0.11404131800927594, -1.632508982793922, -1.5478994251015428, 0.8357076269356797, 1.7688187068473216, 1.264496949776983, -0.5606878737145748, 0.7899350036166821, 0.5549801206147082, -2.41892247118737, -1.1591111874034028, -1.028546523226582, -1.4421569899476774, -0.6359124223550994, 2.2495064986982323, 0.028882216801468907, -1.9267618732373968, -1.2085292182222827, 1.3104993574717918, -0.10207389867499206, -1.0678037205861461, -0.706702236244633, -1.5254042889869504, 1.2866713424244924, -0.8068004292832793, -0.39913846574673073, -1.0397937506217718, -1.630510560085649, -0.7136958969857516, -1.0152867928466245, -0.592639258875477]


model = MLP(8)


model.add_layer(4,'sigmoid')

model = model.compile()

model.from_chromosome(params)

In [9]:
import gymnasium as gym

env = gym.make("LunarLander-v2", render_mode="human")

def run(model):

    observation, info = env.reset()
    racum = 0
    while True:
        action = np.argmax(model.forward(observation))
        observation, reward, terminated, truncated, info = env.step(action)
        
        racum += reward

        if terminated or truncated:
            r = (racum+200) / 500
            return r
        
run(model)

0.9471150654829296

En la siguiente celda se mostrará que el agente es consistente, se mostrará el número de victorias y puntuación media en 1000 iteraciones, no renderizadas por motivos obvios

In [16]:
env = gym.make("LunarLander-v2", render_mode=None)

def run(model):

    observation, info = env.reset()
    racum = 0
    while True:
        action = np.argmax(model.forward(observation))
        observation, reward, terminated, truncated, info = env.step(action)
        
        racum += reward

        if terminated or truncated:
            return racum
        
        
scores = []

for i in range(1000):
    scores.append(run(model))
    
    
scores = np.array(scores)
    
score = np.mean(scores)
wins = scores[scores>=200]

wins = (len(wins)/1000)*100

print(f'Puntuación media en 1000 partidas: {score:.2f}, porcentaje de victorias(más de 200 puntos): {wins:.2f}%')

Puntuación media en 1000 partidas: 273.30, porcentaje de victorias(más de 200 puntos): 98.10%


## Bipedal-Walker

A continuación mostramos el código usado para entrenar la versión normal en SALGA

In [ ]:
model = MLP(24)

# En este caso usamos una capa oculta con 10 neuronas
model.add_layer(10, "relu")
# La función de activación de la salida será la tangente hiperbólica debido a que los valores dados a los motores han de estar entre -1 y 1
model.add_layer(4,'tanh')

model = model.compile()

n_sims = 3 # número de simulaciones por red

env = gym.make("BipedalWalker-v3", render_mode=None, hardcore=False)

def run(model):

    observation, info = env.reset()
    racum = 0
    steps = 0
    while True:
        action = model.forward(observation).reshape(4,)
        observation, reward, terminated, truncated, info = env.step(action)
        
        racum += reward

        if terminated or truncated:
            # No solo entrenamos al robot para llegar al final, le entrenamos para ir lo más rápido posible, quitandole puntuación por steps, hasta un máximo de 160
            racum -= (steps/10)
            r = (racum+400) / 900
            return r
        steps += 1
        
        
def fitness(chromosome):
    
    model.from_chromosome(chromosome)
    
    results = []
    for i in range(n_sims):
        results.append(run(model))
    
    return np.mean(results)

parameters = { 'alphabet':[-2, 2], 'target': 1, 'type':'floating', 'elitism':True, 'norm':True, 'chromsize':model.chromsize(), 'pmut':2/model.chromsize() }

Resultado del entrenamiento anterior se obtienen los siguientes pesos

In [17]:
params = [1.181259307114754, 0.24066104289860815, 0.09594424001269067, 0.23273378295101071, 0.0557602263145181, -0.15154705993919526, 1.0511521587044341, -0.5279510994959336, -1.8272191641178668, -0.06696726053145911, 0.4638721925199422, -0.9963351902752194, 0.6599422206828046, 2.374266264008349, -0.25285171837105724, 2.024016871033684, -0.8292653886683238, -0.917864478131747, -0.018386535686425412, 0.23938911768417576, 0.05865768067419768, 0.28542554647313745, -0.5198991133123673, -0.0654227613408696, -0.30292088908670145, 0.7309922092542616, -1.7461083466468676, -0.5728747760185331, 1.6334734184013096, 1.1672255998390944, 1.229705832384821, -0.6262466421855344, 0.8463725952106651, -0.9973938905538496, 0.8153301206498853, 0.04152464147801148, -0.5912450938809583, -0.17371395854884183, -0.8804600263121249, 1.595269892954158, 2.1245189246108076, 1.6998232305408674, -0.6447107945242555, -0.09942532594069206, -1.0237496972038402, 1.0622972782259978, -0.6575177942232796, -0.9980713541410267, 1.963693060504828, 1.1674993553230109, -0.10101060601498732, -1.6862051085701133, -0.5484037726161674, -0.9747793116165425, 0.5851119712237414, -0.399253912137948, -0.9608516211002656, 0.07151274096104725, 0.056706206227586015, -0.10404891357639746, 1.6900661948216287, 1.2340528604996146, -0.5895015016868264, -0.2590716200174971, -0.5523937964127332, 0.9258835039794492, -0.7641177539327159, 0.24628121232493697, 1.1792595796945349, -1.3849420997520747, 0.5449324308311748, -1.8280884790933434, 0.07233679416575974, 0.021757866658402965, -0.5124603031936216, -0.2879780064489417, 0.5856076579680045, -0.24195510722636482, 1.0590989705639753, 1.952859785426169, -1.4882717922642117, 2.002758592106419, -0.8879832219347494, 0.6163011554556912, 1.1516589720724664, 0.5840775862942923, -0.16934023177117774, 0.8534657621436799, -0.012778408552221444, 0.6826926856453068, 1.4017214351275418, -0.41666853608217513, -0.3000731097380125, 1.970158573406318, 0.42261899882618054, 0.009590196850951442, -0.9485348377980969, 1.1999126141970113, -0.5096692495783898, 0.5401706567734261, 1.6916450761275514, 1.3857523829013974, 0.15136293354572672, 0.9710709154140345, 1.13502086006244, -0.14635211039701812, -0.9579555026779903, 0.5589596404031927, 0.12784361002014216, -1.9213626522882967, -2.3394930881821074, 1.9538047226689754, 0.28475319077822603, -0.6949733785370432, 0.38233689473599003, -0.9597226061920593, 0.09372456054262329, -0.5257572454886866, 0.9771747748308509, 0.5544129256752661, 0.48474997668948483, -0.4925848991086911, 0.3644543443328826, -1.5967062780323362, -0.3583506504354187, -0.9632125105604604, 1.8660932680550442, -0.39407452611672444, 1.1654882226358068, -0.38316801737902284, 0.09105467205206384, -0.3839238713530965, -0.2818137047530863, 0.9463430981472328, -0.8851418377144461, -0.5780019428294134, 0.18329183928705017, -0.001224406372918673, 0.21975261893570652, -0.20115114980885046, 1.9620615323683668, 1.4296485452315661, 0.03841487471629095, -0.5023546248134442, 0.31992817429430837, -0.5396343592893672, 0.3203829141542523, 0.4479771829552286, -1.3611672327679358, 0.20294619959717344, -1.7869621024113063, 0.8804220966687214, 0.006387591288731713, -0.08896793546897852, -0.6838019289900168, -0.6417701982221681, 0.26870287989027397, -0.07541236469008586, 0.39736555063256573, -1.6915634378339393, -1.2523736509498038, -0.4151766682696883, -0.6244368615859509, -0.4864310812292675, -0.40571839990040737, -0.7416388179365124, -1.6923913490423952, 0.1841030610466488, -0.5749652874481973, 1.2153392695011689, -1.7709428024656622, -0.38505069348261184, 0.3887202078090876, -0.20446545151884232, 0.4304618780816766, -0.15429654608413956, -1.2506382318628408, 0.5127087625726431, -2.1520654759865074, -1.0685920627530203, 0.9814967340022958, -0.11470002493080711, 0.3678232294564462, 0.014041582325941946, -0.24460054492849392, -0.6002063325717402, -0.3799564961849871, 0.28036523807968705, -0.8451765640805059, -0.793386470690593, -0.8370298477725157, -0.4326260948541865, 0.3523949881190691, -0.7854268039285521, 0.3842597784038659, 0.2817690149737664, 1.6060799137878015, -0.6873593098700445, -0.2763212367978555, -1.3420465542308861, 1.7886762893988062, 0.5507587062755662, -2.2439465278270716, -0.8677722147856669, -0.8418730143040426, 1.349564599252411, -1.489579107895329, 0.20829653883982208, -1.030843262218273, -0.24612196290935065, -1.3400102428283556, 0.5043082193500514, -0.10860495982777364, -0.9850323692247341, -0.7854859222971429, -0.5561662386351266, -2.325766692955718, 0.31623962744392214, -0.3145113129090903, 0.034853349338863505, -0.301117185843857, -2.091096969798416, -0.4007250880484782, 0.7123356892827826, -0.9900867282418484, -0.1655348931261226, 2.142241775771771, -0.4858773536695396, -1.4195433462844393, 0.6928838061221148, 0.48572359195102305, -0.21700512767392063, -0.45693690978974494, 0.580022845407809, -0.1373430311693879, -0.9734433271036714, 1.5475348754018075, -0.04281791707320996, 0.7766179034643955, -0.6944577170441147, -1.3813636216953487, -0.5417027512351384, -1.4638987420315939, -0.49977433465534804, -0.8748723553319611, -0.24350036096407038, -0.3890030616867598, -1.4704128953256153, -1.3795518965050595, 0.7775830654178123, -1.373484964602578, 1.5654369721801522, -0.1998158503774262, 1.8386687498339183, -1.8706323660677413, -1.7222942232062366, -0.12547424925721692, -0.2784318224539401, 8.921627734442827e-06, -2.1262596185288447, 0.3262801760891293, 0.8485347200937831, 2.7137581462487823, -0.9307532809992036, -0.44911250615982473, -0.35139673042932945, 0.46913996518760415, -0.2031923263313757, -0.16805515195166834, -0.10500150881538463, -0.3040093696512096, -1.198357403143699, 0.3913349336721167, 1.5261790314428132, 2.088444324366079, 1.3551537549361818, 0.11874958053119324, 0.8153656301552006, -0.0605274432876109, 1.1040053355666246, -0.9248387124595976, -0.07254919813447482, 0.03347540965747606, -2.75883370963182, -0.007363404985387398, 1.0098286640967684, -1.8639014954400275, -0.6015658589080491, -0.020412586205679886, 0.05289223038261217, 1.6620967019933583, -1.8023588577550222, 0.026504910401395878, -1.8565845658750446]

model = MLP(24)

model.add_layer(10,"relu")
model.add_layer(4,'tanh')

model = model.compile()

model.from_chromosome(params)

Demostración de la validez del modelo

In [19]:
env = gym.make("BipedalWalker-v3", render_mode="human", hardcore=False)

def run(model):

    observation, info = env.reset()
    racum = 0
    steps = 0
    while True:
        
        action = model.forward(observation).reshape(4,)
        observation, reward, terminated, truncated, info = env.step(action)
            
        racum += reward

        if terminated or truncated:
            r = (racum+200) / 500
            return racum, steps
            


        steps+=1
        
run(model)

(276.2324270012772, 1263)

In [25]:
env = gym.make("BipedalWalker-v3", render_mode=None, hardcore=False)

scores = []
steps = []

for i in range(1000):
    score, step = run(model)
    scores.append(score)
    steps.append(step)

score = np.mean(scores)
step = np.mean(steps)

scores = np.array(scores)
steps = np.array(steps)


print(f'Puntuación media en 1000 intentos: {score:.2f}, en {step:.2f} steps')


Puntuación media en 1000 intentos: 262.84, en 1192.09 steps


No hemos conseguido superar el Bipedal-Walker en su versión hardcore, pero expondremos nuestros intentos.

En primer lugar intentamos entrenar una red directamente para el hardcore, no obtuvimos resultados, intentamos agrandar la red, pero salga no acepta cromosomas excesivamente grandes.

Entrenamos una red de 20 neuronas en su capa oculta para le versión normal, guardando su población para posteriormente cargarla en la versión hardcore, el robot aprendio a quedarse parado o bien si le penalizabamos mucho por aguantar todos los steps, aprendió a suicidarse.

Por lo que intentamos evitar esto, modificando el reward, sin exito

In [ ]:
def run(model):

    observation, info = env.reset()
    racum = 0
    # Guardamos las acciones que el agente ha llevado a cabo
    actions = []
    steps = 0
    aux = 1
    while True:
        
        action = model.forward(observation).reshape(4,)
        observation, reward, terminated, truncated, info = env.step(action)
        
        # Las acciones se guardan a partir del step 100
        if steps >100:
            actions.append(action)
            # Calculamos la distancia de nuestra acción a la media de las anteriores
            aux += np.sqrt(np.sum(np.square(action - np.mean(actions, axis=0))))
        
        # Establecemos que como máximo comparamos con las 5 acciones anteriores
        if len(actions) ==5:
            actions.pop(0)
            
        racum += reward

        if terminated or truncated:
            racum -= (steps/10)
            r = (racum+200) / 500
            
            
            # Penalizamos cuanto menor sea la distancia entre acciones
            if steps > 100:
                r-= 0.01/(aux/steps)
                
            return r

        steps+=1

Probamos otros métodos, usando varias poblaciones y especiación, usando librerias externas, dado que salga no da esta opción, tampoco tuvimos exito